In [ ]:
!pip install yt-dlp moviepy whisper transformers
!apt-get install ffmpeg
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 16.7 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=b48a348eb6cd7dc00b7baad663931c5ca63ce18d9b734cf2170da3bbf782d16a
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml)

In [ ]:
import whisper
from transformers import pipeline
from moviepy.editor import VideoFileClip
import yt_dlp
import os
import torch

# Função para baixar o vídeo usando yt-dlp
def download_video(url):
    ydl_opts = {
        'format': 'bestvideo+bestaudio',
        'outtmpl': 'video.%(ext)s',
        'merge_output_format': 'mp4',  # Força a saída em formato mp4
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',  # Converte o vídeo para mp4 após o download
        }],
        'keepvideo': True,  # Mantém os arquivos originais
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    if os.path.exists('video.mp4'):
        print("Vídeo baixado com sucesso.")
        return 'video.mp4'
    else:
        raise FileNotFoundError("Erro: o arquivo 'video.mp4' não foi encontrado após o download.")

# Função para extrair o áudio do vídeo
def extract_audio(video_path, audio_output):
    if os.path.exists(video_path):
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_output)
        print("Áudio extraído com sucesso.")
    else:
        raise FileNotFoundError(f"Erro: o arquivo '{video_path}' não foi encontrado.")

# Função para transcrever o áudio
def transcribe_audio(audio_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("base").to(device)
    result = model.transcribe(audio_path)
    print("Transcrição concluída.")
    return result['text']

# Função para dividir o texto em partes menores
def split_text(text, max_length):
    words = text.split()
    current_length = 0
    current_chunk = []

    for word in words:
        current_length += len(word) + 1  # +1 para o espaço
        if current_length > max_length:
            yield ' '.join(current_chunk)
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        yield ' '.join(current_chunk)

# Função para resumir o texto
def summarize_text(text):
    summarizer = pipeline("summarization")
    summaries = []
    for chunk in split_text(text, 512):  # Certificar que cada chunk tem no máximo 512 tokens
        summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    print("Resumos concluídos.")
    return ' '.join(summaries)

# Entrada do usuário: URL do vídeo
video_url = input("Digite o URL do vídeo: ")

# 1. Baixar o vídeo
video_path = download_video(video_url)
audio_path = "audio.wav"

# 2. Extrair o áudio
extract_audio(video_path, audio_path)

# 3. Transcrever o áudio
transcription = transcribe_audio(audio_path)
print("Transcrição: ", transcription)

# 4. Resumir a transcrição
summary = summarize_text(transcription)
print("Resumo: ", summary)

# Remover arquivos temporários (opcional)
os.remove(video_path)
os.remove(audio_path)


  if event.key is 'enter':



Digite o URL do vídeo: https://www.youtube.com/watch?v=ex445LR3obA
[youtube] Extracting URL: https://www.youtube.com/watch?v=ex445LR3obA
[youtube] ex445LR3obA: Downloading webpage
[youtube] ex445LR3obA: Downloading ios player API JSON
[youtube] ex445LR3obA: Downloading mweb player API JSON
[youtube] ex445LR3obA: Downloading player 6f0e2ed6
[youtube] ex445LR3obA: Downloading m3u8 information
[info] ex445LR3obA: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 36
[download] Destination: video.f616.mp4
[download] 100% of   35.28MiB in 00:00:05 at 6.22MiB/s                 
[download] Destination: video.f251.webm
[download] 100% of    2.40MiB in 00:00:00 at 13.35MiB/s  
[Merger] Merging formats into "video.mp4"
[VideoConvertor] Not converting media file "video.mp4"; already is in target format mp4
Vídeo baixado com sucesso.
MoviePy - Writing audio in audio.wav


MoviePy - Done.
Áudio extraído com sucesso.


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 80.3MiB/s]
  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Transcrição concluída.
Transcrição:   Boa noite, eu pensei numa ideia pra gente apresentar pro Jucelino. Que débil. Gente, a gente... A gente tem que sair daqui pra longe. Quando você diz, é, saindo, a gente vai dar a área jada. Não, não, Verna. É, a gente realmente se mudar de vez. mudar o governo daqui do Rio pra um lugar, quando ninguém vai encher o nosso saco. Esse é um sonho. Tem balia. Caralho, vai já ouvir o falar. É, tem sol, tem marco. Não dá, não dá porque a Bahia ajuda toda a tomada por paulista. Trancoso, uma espécie de filial, e tá em bebê. Não faz sentido pra lá. A gente vai, e meia horas, começa um panelazo. E, bem, repente, o sul. Fui agora. Clima, mais amino, mais fresco, queijo raspe, esmano. É, mas... precisa ser realmente dentro do Brasil. Não faz sentido da capital, em outro país, não dá. Pode estar, será, agora é um arrobo da minha parte. Mas eu tenho cita-araras. Tem os lotes ali por perto. Você dá pra comprar ali? Isso é secretário, não é verdade. Mas dá pra gen

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

  warnings.warn(

Your max_length is set to 130, but your input_length is only 60. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


Resumos concluídos.
Resumo:   A gente realmente se mudar de vez. mudar o governo daqui do Rio pra um lugar. Que débil. Gente, a gente... A gentente tem que sair daquir pra longe. Quando você diz, é, saindo, a . gente vai dar a área jada. Não, não dá porque a Bahia ajuda toda a tomada por paulista.  Não faz sentido pra lá. A gente vai, e meia horas, começa um panelazo. Eu pensei em goyares. Goyares is meio fora de mão, né? Eu tenho cita-araras. Pode estar, será, agora .  Não te sabia nem that existia nem que existia, o que é goyares? Goyares is a secretário, né? Vamos supor que uma pessoa te robe muito dinheiro .  O goyares is o mais longe que dá pra ir antes de virar a Mato. do Mato . Eventualmente, sem elas do lado, vamos acabar com prostitutas. E como estaremos sozinhos, a gente vai poder. Tá, mas há como a prostituta diferente por dia, né?  Eu pensei em chamar um lugar de lasvegas brasileiras . Dá muito na cara, gente precisa pensar no homem pouco atrativo . Existe um vão de ileto p